# Data Conversion of SBE 911plus .hex data
This python notebook shows how to convert a .hex file into pandas dataframe with scientific values.

Please contact [SBS customer support](https://www.seabird.com/support) for help or to request additional features.

Example data provided by CalCOFI. Station Santa Barbara Basin

## Init

The initialization code section below is used to import required libraries.

In [ ]:
# Native imports
import os

# Third-party imports
import numpy as np
import pandas as pd
import gsw

# Sea-Bird imports
import seabirdscientific.conversion as conv
import seabirdscientific.instrument_data as id
import seabirdscientific.visualization as viz
import example_data.example_coefficients as ec

## [Data Conversion](#proc-list)

This section shows how to convert raw data contained in a .hex file into scientific units for the instruments that follow:  
- 9/11plus

In [ ]:
hex_file = os.path.join("example_data", "SBE911plus", "2507054.hex")

# Convert raw hexadecimal string to raw frequencies
raw_data = id.read_hex_file(
    filepath=hex_file,
    instrument_type=id.InstrumentType.SBE911Plus,
    enabled_sensors=[
        id.Sensors.Temperature,
        id.Sensors.Conductivity,
        id.Sensors.Pressure,
        id.Sensors.SecondaryTemperature,
        id.Sensors.SecondaryConductivity,
        id.Sensors.ExtVolt0,
        id.Sensors.ExtVolt1,
        id.Sensors.ExtVolt2,
        id.Sensors.ExtVolt3,
        id.Sensors.ExtVolt4,
        id.Sensors.ExtVolt5,
        id.Sensors.ExtVolt6,
        id.Sensors.ExtVolt7,
        id.Sensors.SPAR,
        id.Sensors.nmeaLocation,
        id.Sensors.SystemTime,
    ],
    frequency_channels_suppressed=0,
    voltage_words_suppressed=0,
)

In [ ]:
# Convert raw frequencies to scientific values
sample_interval = 1 / 24

temperature = conv.convert_temperature_frequency(
    frequency=raw_data["temperature"].values,
    coefs=ec.temperature_coefs_sn5102,
    standard="ITS90",
    units="C",
)

pressure_dbar = conv.convert_pressure_digiquartz(
    pressure_count=raw_data["digiquartz pressure"].values,
    compensation_voltage=raw_data["temperature compensation"],
    coefs=ec.pressure_coefs_sn0936,
    units="dbar",
    sample_interval=sample_interval,
)

conductivity = conv.convert_conductivity(
    conductivity_count=raw_data["conductivity"].values,
    temperature=temperature,
    pressure=pressure_dbar,
    coefs=ec.conductivity_coefs_sn3569,
    scalar=0.1,
)

secondary_temperature = conv.convert_temperature_frequency(
    frequency=raw_data["temperature"].values,
    coefs=ec.temperature_coefs_sn5109,
    standard="ITS90",
    units="C",
)

secondary_conductivity = conv.convert_conductivity(
    conductivity_count=raw_data["conductivity"].values,
    temperature=temperature,
    pressure=pressure_dbar,
    coefs=ec.conductivity_coefs_sn2206,
    scalar=0.1,
)

salinity = gsw.SP_from_C(
    C=conductivity / 1000,  # mS/cm
    t=temperature,  # ITS90 C
    p=pressure_dbar,  # dbar
)

chlorophyll = conv.convert_eco(raw=raw_data["volt 1"], coefs=ec.chlorophyll_a_coefs_sn3122)

height = conv.convert_altimeter(volts=raw_data["volt 2"], coefs=ec.altimeter_coefs_sn46604)

oxygen = conv.convert_sbe43_oxygen(
    voltage=raw_data["volt 4"],
    temperature=temperature,
    pressure=pressure_dbar,
    salinity=salinity,
    coefs=ec.oxygen_43_coefs_sn1590,
    apply_tau_correction=True,
    apply_hysteresis_correction=True,
    window_size=1,
    sample_interval=sample_interval,
)

oxygen_secondary = conv.convert_sbe43_oxygen(
    voltage=raw_data["volt 5"],
    temperature=temperature,
    pressure=pressure_dbar,
    salinity=salinity,
    coefs=ec.oxygen_43_coefs_sn0680,
    apply_tau_correction=True,
    apply_hysteresis_correction=True,
    window_size=1,
    sample_interval=1 / 24,
)

ph = conv.convert_sbe18_ph(
    raw_ph=raw_data["volt 7"], temperature=temperature, coefs=ec.ph_coefs_sn0709
)

spar = conv.convert_spar_biospherical(volts=raw_data["surface par"], coefs=ec.spar_coefs_sn20659)


# Flag to be used in data processing
flag = np.zeros(len(temperature))

data = pd.DataFrame(
    {
        "temp": temperature,
        "cond": conductivity,
        "press": pressure_dbar,
        "temp_secondary": secondary_temperature,
        "cond_secondary": secondary_conductivity,
        "chlorophyll": chlorophyll,
        "height": height,
        "oxygen": oxygen,
        "oxygen_secondary": oxygen_secondary,
        "ph": ph,
        "spar": spar,
        "nmea_lat": raw_data["NMEA Latitude"],
        "nmea_long": raw_data["NMEA Longitude"],
        "pump_status": raw_data["SBE911 pump status"],
        "bottom_contact": raw_data["SBE911 bottom contact status"],
        "confirm_status": raw_data["SBE911 confirm status"],
        "modem_status": raw_data["SBE911 modem status"],
        "system_time": raw_data["system time"],
        "data_integrity": raw_data["data integrity"],
        "flag": flag,
    }
)

data

## [Data Plotting](#proc-list)

In [ ]:
config = viz.ChartConfig(
    title="9/11plus Data Conversion",
    x_names=["temp", "cond", "chlorophyll", "oxygen"],
    y_names=["press"],
    z_names=[],
    chart_type="overlay",
    plot_loop_edit_flags=False,
    lift_pen_over_bad_data=True,
)

chart_data = viz.ChartData(data, config)
fig = viz.plot_xy_chart(chart_data, config)

# plotly customizations
fig["layout"]["yaxis"]["autorange"] = "reversed"
fig.data[0].name = "Temperature"
fig.data[1].name = "Conductivity"
fig.data[2].name = "Chlorophyll"
fig.data[3].name = "Oxygen"
fig["layout"]["yaxis"]["title"] = "Pressure [dbar]"
fig["layout"]["xaxis"]["title"] = "Temperature [ITS-90 degrees C]"
fig["layout"]["xaxis2"]["title"] = "Conductivity [S/m]"
fig["layout"]["xaxis3"]["title"] = "Chlorophyll"
fig["layout"]["xaxis4"]["title"] = "Oxygen [ml/l]"

fig.update_layout(height=800)
fig.show()

## Processing

This converted data should now be processed using the tools in processing.ipynb to produce a final data product.